## MultiOn Demo

This notebook walks through an example of using LlamaIndex with MultiOn to browse the web on the users behalf.

First, we import the OpenAIAgent that will control the Multion session:

In [ ]:
# Set up OpenAI
import openai
from llama_index.agent import OpenAIAgent

openai.api_key = "sk-your-key"

We then import the MultiOn tool and initialize our agent with the tool. When the MultiOn tool is initialized, the user will be prompted to authenticate the session in their browser

In [ ]:
# Set up MultiOn tool
from llama_index.tools.multion.base import MultionToolSpec

multion_tool = MultionToolSpec()

Already logged in.


To support the MultiOn browsing session, we will also give our LlamaIndex agent a tool to search and summarize a users gmail inbox. We set up that tool below. For more information on the gmail tool, see the [Gmail notebook here](https://github.com/emptycrown/llama-hub/blob/main/llama_hub/tools/notebooks/gmail.ipynb).

We will use this tool later on to allow the agent to gain more context around our emails

In [ ]:
# Import and initialize our tool spec
from llama_index.tools.gmail.base import GmailToolSpec
from llama_index.tools.ondemand_loader_tool import OnDemandLoaderTool

# Initialize the Gmail tool to search our inbox
gmail_tool = GmailToolSpec()

# Wrap the tool so we don't overflow the main Agent's context window
gmail_loader_tool = OnDemandLoaderTool.from_tool(
    gmail_tool.to_tool_list()[1],
    name="gmail_search",
    description="""
        This tool allows you to search the users gmail inbox and give directions for how to summarize or process the emails

        You must always provide a query to filter the emails, as well as a query_str to process the retrieved emails.
        All parameters are required
        
        If you need to reply to an email, ask this tool to build the reply directly
        Examples:
            query='from:adam subject:dinner', max_results=5, query_str='Where are adams favourite places to eat'
            query='dentist appointment', max_results=1, query_str='When is the next dentist appointment'
            query='to:jerry', max_results=1, query_str='summarize and then create a response email to jerrys latest email'
            query='is:inbox', max_results=5, query_str='Summarize these emails'
        """,
)

In [ ]:
# Initialize our Agent with the MultiOn and Gmail loader tool
agent = OpenAIAgent.from_tools(
    [*multion_tool.to_tool_list(), gmail_loader_tool],
    system_prompt="""
    You are an AI agent that assists the user in crafting email responses based on previous conversations.
    
    The gmail_search tool connects directly to an API to search and retrieve emails, and answer questions based on the content.
    The browse tool allows you to control a web browser with natural language to complete arbitrary actions on the web.
    
    Use these two tools together to gain context on past emails and respond to conversations for the user.
    """,
    verbose=True,
)

Our agent is now set up and ready to browse the web!

In [ ]:
print(agent.chat("browse to the latest email from adam and open the email"))

=== Calling Function ===
Calling function: browse with args: {
  "instruction": "browse to the latest email from Adam and open the email"
}
running new session
Server Disconnected. Please press connect in the Multion extension popup
Got output: {'url': 'https://mail.google.com/mail/u/0/', 'status': 'CONTINUE', 'action_completed': 'I am navigating to Gmail to find the latest email from Adam.\n\n', 'content': 'Mail\n\nChat\n\nfo}\nrR\n\nSpaces\n\nOr\n\nMeet\n\n4 Gmail\n\n2 Compose\nfa Inbox\n\nyx © Starred\n© _ Snoozed\nE& Sent\n\nvy More\nLabels\n\nQ Search in mail iz\nOr ¢G i\n\nAdam, me 5\n\nMedium\n\nMedium\n\nEnable desktop notifications for Hypotenuse Mail.\n\nMedium Daily Digest\nLinda Chen\n\nCanva\n\nZapier\n\nSlava Timbaliuc\nSlava Timbaliuc\nSlava Timbaliuc\nMedium Daily Digest\nSlava Timbaliuc\nMedium - Programmi.\n1Password\n\nYelp San Francisco\nMedium Daily Digest\nLucid\n\nDeb at Zapier\n1Password\n\nMedium Daily Digest\nsimon\n\nMedium Daily Digest\n\n@rctver @ @ 3 Googl

In [ ]:
print(
    agent.chat(
        "Summarize the email chain with adam and create a response to the last email"
        " that confirms all the details"
    )
)

=== Calling Function ===
Calling function: gmail_search with args: {
  "query": "from:adam",
  "max_results": 5,
  "query_str": "Summarize the email chain with Adam"
}
Got output: The email chain with Adam consists of two emails. In the first email, the sender asks about the soonest delivery date for a chair and proposes a price of $475 with cash payment. In the second email, Adam agrees to the price of $500 for the chair and delivery, provides his address and phone number, and thanks the sender.
Based on the summary of the email chain with Adam, here is a response that confirms all the details:

"Hi Adam,

Thank you for your email. I wanted to confirm that I have received your message regarding the chair. I agree to the price of $500 for the chair and delivery. 

Here are the details:
- Price: $500
- Delivery address: 1513 St. Paul Street, St Catharines, Ontario, L2F
- Phone number: 905-328-6945

I appreciate your prompt response and I look forward to receiving the chair on Monday as 

In [ ]:
print(
    agent.chat(
        "pass the entire generated email to the browser and have it send the email as a"
        " reply to the chain"
    )
)

=== Calling Function ===
Calling function: browse with args: {
  "instruction": "compose a reply email and paste the following content:\n\n\"Hi Adam,\n\nThank you for your email. I wanted to confirm that I have received your message regarding the chair. I agree to the price of $500 for the chair and delivery.\n\nHere are the details:\n- Price: $500\n- Delivery address: 1513 St. Paul Street, St Catharines, Ontario, L2F\n- Phone number: 905-328-6945\n\nI appreciate your prompt response and I look forward to receiving the chair on Monday as discussed.\n\nBest regards,\n[Your Name]\""
}
session updated
Got output: {'url': 'https://mail.google.com/mail/u/0/', 'status': 'CONTINUE', 'action_completed': 'I am clicking on the "Reply" button to start composing the reply email to Adam Hofmann.\n\n', 'content': 'Mail\n\nChat\n\nfo}\nrR\n\nSpaces\n\nOr\n\nMeet\n\nM4 Gmail Q. Search in mail iz @rctver @ } 3: Google 6\n\nG Compose € Oo w Y © G&G mo D : 10f1928 << >\n2\nfb Inbox\nyr Starred Ca] Adam H